# Product Recommendation
Reference: https://ieeexplore.ieee.org/document/5430993

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

In [2]:
tf.__version__

'2.3.1'

## Data Preprocessing

In [3]:
Y_data = pd.read_csv('data/Y.csv', header=None, names=['Rating','Movie','User'], dtype=np.int32) # training data
P_data = pd.read_csv('data/P.csv', header=None, names=['Rating','Movie','User'], dtype=np.int32) # test data ('probe-set' mentioned in paper)

In [4]:
display(Y_data.head())
display(P_data.head())
Y_data.shape, P_data.shape

,Rating,Movie,User
0,5,2,1
1,4,7,1
2,4,8,1
3,4,11,1
4,4,12,1


,Rating,Movie,User
0,3,6,1
1,5,96,1
2,3,1,2
3,3,33,3
4,5,42,4


((3399874, 3), (189699, 3))

In [5]:
print(Y_data['Rating'].max(), Y_data['Movie'].max(), Y_data['User'].max())
print(P_data['Rating'].max(), P_data['Movie'].max(), P_data['User'].max())

5 100 137328
5 100 137328


In [6]:
k, n = Y_data['Movie'].max(), Y_data['User'].max()
k, n

(100, 137328)

In [7]:
def generate_indices_pair_list(data):
    user_id = 1
    indices_list = list()
    for index, row in enumerate(tqdm(data, total=data.shape[0])):
        if row[2] != user_id:
            user_id = row[2]
            indices_list.append((user_id - 1, index - 1))
    indices_list.append((user_id, index)) # append the last user id and ending index pair

    indices_pair_list = list()
    for (user_id, index_ending) in indices_list:
        if index_ending == indices_list[0][1]: # if the first ending index
            indices_pair_list.append((1, 0, index_ending))
        else:
            index_beginning = indices_pair_list[-1][2] + 1
            indices_pair_list.append((user_id, index_beginning, index_ending))
    return indices_pair_list

In [8]:
Y_data = Y_data.values
P_data = P_data.values

indices_pair_list_Y_data = generate_indices_pair_list(Y_data)
indices_pair_list_P_data = generate_indices_pair_list(P_data)
len(indices_pair_list_Y_data) == len(indices_pair_list_P_data)

100%|██████████| 189699/189699 [00:00<00:00, 365474.89it/s]


True

In [9]:
data_preprocessed = list()
for index_pair_Y_data, index_pair_P_data in tqdm(zip(indices_pair_list_Y_data, indices_pair_list_P_data), total=len(indices_pair_list_Y_data)):
    Y_data_t = Y_data[index_pair_Y_data[1]:index_pair_Y_data[2]+1, :]
    
    movie_ids_t_indices = Y_data_t[:, 1] - 1    
    H_yt = tf.constant(np.identity(k)[movie_ids_t_indices], dtype=tf.float32)
    H_xt = tf.constant(np.delete(np.identity(k), movie_ids_t_indices, 0), dtype=tf.float32)    
    k_t = tf.constant(H_yt.shape[0], dtype=tf.float32)
    
    z_t_st_indices = np.vstack((movie_ids_t_indices, np.zeros(movie_ids_t_indices.shape[0], dtype=np.int64))).T
    z_t = tf.sparse.to_dense(tf.SparseTensor(indices=z_t_st_indices, values=Y_data_t[:, 0].astype(np.float32), dense_shape=[k, 1]))
    
    y_t = tf.matmul(H_yt, z_t)
    
    P_data_t = P_data[index_pair_P_data[1]:index_pair_P_data[2]+1, :]
    movie_ids_t_P_data = P_data_t[:, 1]
    ratings_t_P_data = tf.expand_dims(P_data_t[:, 0].astype(np.float32), axis=1)
    data_preprocessed.append((H_yt, H_xt, tf.transpose(H_yt), tf.transpose(H_xt), k_t, y_t, movie_ids_t_P_data, ratings_t_P_data))
        
del Y_data
del P_data

100%|██████████| 137328/137328 [01:23<00:00, 1644.25it/s]


## Initialization

$\mu$ has 1 type available

$N = \sum_{t=1}^{n}H_{y_t}'H_{y_t}$

$\hat{\mu}^0 = N^{-1}\sum_{t-1}^{n}H_{y_t}'y_{t}$

In [10]:
# initial estimate of mu
N = 0
H_yty_t = 0
    
for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, y_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
    N += tf.matmul(H_yt_trans, H_yt)
    H_yty_t += tf.matmul(H_yt_trans, y_t)

100%|██████████| 137328/137328 [00:09<00:00, 14233.35it/s]


In [11]:
# The ith diagonal element of N equals the total number of ratings of the ith product.
N

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[20017.,     0.,     0., ...,     0.,     0.,     0.],
       [    0., 23917.,     0., ...,     0.,     0.,     0.],
       [    0.,     0., 31634., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ..., 60896.,     0.,     0.],
       [    0.,     0.,     0., ...,     0., 61521.,     0.],
       [    0.,     0.,     0., ...,     0.,     0., 64506.]],
      dtype=float32)>

In [12]:
mu_hat0 = tf.matmul(tf.linalg.inv(N), H_yty_t)
tf.transpose(mu_hat0)

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[3.4526653, 3.5767446, 3.2878866, 3.9047875, 3.7903547, 3.4441562,
        3.1907156, 4.5283504, 3.8201375, 3.6159503, 3.4038272, 3.8372512,
        4.076039 , 4.228367 , 3.3539548, 4.0645275, 3.7211962, 3.4870086,
        4.1638894, 3.4098244, 3.86926  , 3.435835 , 3.2032444, 4.084879 ,
        3.2319984, 3.886648 , 4.331895 , 4.3835816, 4.316387 , 3.8659174,
        4.339757 , 3.8914788, 3.7002926, 3.3624778, 4.3290153, 4.0670686,
        4.56922  , 3.771041 , 3.6858668, 3.8453238, 4.3454113, 3.9099922,
        3.3994992, 3.607868 , 3.962671 , 4.143861 , 3.4072049, 3.7040226,
        4.0035033, 4.6428022, 3.216233 , 3.7723858, 4.265651 , 4.453773 ,
        3.8384895, 3.793742 , 3.762917 , 3.886986 , 3.8004174, 4.34697  ,
        3.8046958, 3.8462481, 3.641226 , 3.2722168, 3.4233348, 3.7163155,
        3.2069893, 4.4541044, 4.265413 , 3.8610919, 4.483009 , 4.3621464,
        3.5388138, 4.1171074, 3.8946984, 3.3607926, 4.1794305,

R has 4 types available

$R_{1} = I$

$R_{2} = N^{-1}diag(S)$

$R_{3} = diag(S)^{-1/2}Sdiag(S)^{-1/2}$

$R_{4} = N^{-1/2}SN^{-1/2}$

where $S = \sum_{t=1}^{n}H_{y_{t}}'(y_t - H_{y_{t}}\hat{\mu}^0)(y_t - H_{y_{t}}\hat{\mu}^0)'H_{y_{t}}$

In [13]:
# initial estimates of R (4 types available)
R_hat0_1 = tf.eye(k, dtype=tf.float32)
R_hat0_1

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [14]:
S = 0
for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, y_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
    Hytmu_hat0 = tf.matmul(H_yt, mu_hat0)
    S += H_yt_trans @ (y_t - Hytmu_hat0) @ tf.transpose(y_t - Hytmu_hat0) @ H_yt

100%|██████████| 137328/137328 [00:19<00:00, 7185.78it/s]


In [15]:
# diag_S is the diagonal matrix consisting of the diagonal elements of S
N_inv = tf.linalg.inv(N)
diag_S = tf.linalg.diag(tf.linalg.tensor_diag_part(S))
R_hat0_2 = tf.matmul(N_inv, diag_S)
R_hat0_2

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[1.724364  , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.94215506, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.4365153 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.1832098 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.0349715 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.2620207 ]], dtype=float32)>

In [16]:
# R_hat0_3 is not a good initializer when rating variances are far from one
diag_S_inv = tf.linalg.inv(diag_S)
R_hat0_3 = tf.linalg.sqrtm(diag_S_inv) @ S @ tf.linalg.sqrtm(diag_S_inv)
R_hat0_3

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[ 1.        ,  0.07418474, -0.01158332, ..., -0.01462824,
        -0.02215266, -0.01845053],
       [ 0.07418475,  1.        ,  0.03674483, ...,  0.02561698,
         0.03563043,  0.03926769],
       [-0.01158332,  0.03674483,  0.9999999 , ...,  0.10954399,
         0.12823027,  0.1556249 ],
       ...,
       [-0.01462824,  0.02561698,  0.10954399, ...,  1.        ,
         0.19780062,  0.15164363],
       [-0.02215266,  0.03563043,  0.12823027, ...,  0.19780062,
         1.0000001 ,  0.18997027],
       [-0.01845053,  0.03926768,  0.1556249 , ...,  0.15164362,
         0.18997027,  1.        ]], dtype=float32)>

In [17]:
R_hat0_4 = tf.linalg.sqrtm(N_inv) @ S @ tf.linalg.sqrtm(N_inv)
R_hat0_4

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[ 1.724364  ,  0.09455624, -0.01823068, ..., -0.02089477,
        -0.02959406, -0.02721802],
       [ 0.09455624,  0.94215494,  0.04274768, ...,  0.02704705,
         0.03518409,  0.04281832],
       [-0.01823068,  0.04274768,  1.4365153 , ...,  0.14281526,
         0.15635432,  0.20954023],
       ...,
       [-0.02089477,  0.02704705,  0.14281526, ...,  1.1832099 ,
         0.21888837,  0.18530548],
       [-0.02959407,  0.03518409,  0.15635432, ...,  0.21888836,
         1.0349715 ,  0.2171116 ],
       [-0.02721801,  0.04281832,  0.20954023, ...,  0.18530548,
         0.2171116 ,  1.2620206 ]], dtype=float32)>

## Expectation Maximization Algorithm

1. $\hat{R}^{i+1} = \frac{1}{n} \sum_{t=1}^{n} (\hat{Z}_t - \mu)(\hat{Z}_t - \mu)' + H_{x_t}' \bigg(\hat{R}_{x_t}^{i} - \hat{R}_{x_ty_t}^{i} \big(\hat{R}_{y_t}^{i}\big)^{-1} \big(\hat{R}_{x_ty_t}^{i}\big)'\bigg) H_{x_t}$ <br />
where <br />
$\hat{Z} = H_{y_t}'y_t + H_{x_t}'\hat{X_t}$ <br />
$\hat{X_t} = R_{x_ty_t} R_{y_t}^{-1} (y_t - \mu_{y_t}) + \mu_{x_t}$ <br />
$R_{x_t} = H_{x_t} R H_{x_t}'$ <br />
$R_{y_t} = H_{y_t} R H_{y_t}'$ <br />
$R_{x_ty_t} = H_{x_t}RH_{y_t}'$ <br />
$\mu_{x_t} = H_{x_t}\mu$ <br />
$\mu_{y_t} = H_{y_t}\mu$

2. $\hat{\mu} = \big(\sum_{t=1}^{n}H_{y_t}'R_{y_t}^{-1}H_{y_t}\big)^{-1} \big(\sum_{t=1}^{n}H_{y_t}'R_{y_t}^{-1}y_t\big)$

In [18]:
LOG_2PI = tf.math.log(2*tf.constant(np.pi, dtype=tf.float32))

@tf.function(experimental_relax_shapes=True)
def run_graph_em(mu, R, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans, k_t):
    # for R estimation
    R_xt = H_xt @ R @ H_xt_trans
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_det = tf.linalg.det(R_yt)
    R_yt_inv = tf.linalg.inv(R_yt)
    R_xtyt = H_xt @ R @ H_yt_trans

    mu_yt = tf.matmul(H_yt, mu)
    mu_xt = tf.matmul(H_xt, mu)

    X_t_hat = R_xtyt @ R_yt_inv @ (y_t - mu_yt) + mu_xt
    Z_t_hat = H_yt_trans @ y_t + H_xt_trans @ X_t_hat
    
    R_hat_sum_part = (Z_t_hat - mu) @ tf.transpose(Z_t_hat - mu) \
                        + H_xt_trans @ (R_xt - R_xtyt @ R_yt_inv @ tf.transpose(R_xtyt)) @ H_xt

    # for mu estimation
    Hyt_trans_Ryt_inv_Hyt_sum_part = H_yt_trans @ R_yt_inv @ H_yt
    Hyt_trans_Ryt_inv_yt_sum_part = H_yt_trans @ R_yt_inv @ y_t
    
    # for log likelihood calculation
    log_p_hat_part = -1/2*(tf.math.log(R_yt_det) + tf.transpose(y_t - mu_yt) @ R_yt_inv @ (y_t - mu_yt) + k_t*LOG_2PI) 
    
    return R_hat_sum_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part

In [19]:
def expectation_maximization(mu, R):
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    R_hat_sum = 0
    log_p_hat = 0
    
    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, y_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
        R_hat_sum_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part = run_graph_em(mu, R, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans, k_t)
        
        R_hat_sum += R_hat_sum_part
        Hyt_trans_Ryt_inv_Hyt_sum += Hyt_trans_Ryt_inv_Hyt_sum_part
        Hyt_trans_Ryt_inv_yt_sum += Hyt_trans_Ryt_inv_yt_sum_part
        log_p_hat += log_p_hat_part
        
    R_hat = R_hat_sum / n
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)    
    return mu_hat, R_hat, log_p_hat

In [20]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = tf.constant(-np.inf, dtype=tf.float32)

for i in range(30):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = expectation_maximization(mu, R)
    convergence_criterion = log_p_hat/n - log_p/n < delta
    
    print(f'normalized log_p_hat: {(log_p_hat/n).numpy().flatten()[0]:.5}')
    print(f'normalized log_p:     {(log_p/n).numpy().flatten()[0]:.5}')
    print(f'convergence gap:      {(log_p_hat/n - log_p/n).numpy().flatten()[0]:.5}')
    
    if convergence_criterion:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 1/137328 [00:00<7:26:40,  5.12it/s]

iteration: 0


  0%|          | 146/137328 [00:00<01:34, 1454.28it/s]

normalized log_p_hat: -32.233
normalized log_p:     -inf
convergence gap:      inf


  0%|          | 172/137328 [00:00<01:19, 1715.44it/s]

...

normalized log_p_hat: -31.415
normalized log_p:     -31.415
convergence gap:      0.00057983


  0%|          | 156/137328 [00:00<01:28, 1554.32it/s]

normalized log_p_hat: -31.414
normalized log_p:     -31.415
convergence gap:      0.00057793
iteration: 25


100%|██████████| 137328/137328 [01:30<00:00, 1517.27it/s]

normalized log_p_hat: -31.414
normalized log_p:     -31.414
convergence gap:      0.00049591


In [21]:
# 26 iterations, ~38 min
np.save('results/em_mu.npy', mu_hat)
np.save('results/em_R.npy', R_hat)
np.save('results/em_log_p.npy', log_p_hat)

## McMichael’s Algorithm

1. $\hat{R}^{i+1} = \hat{R}^{i} + \gamma \hat{R}^{i} \bigg(\frac{d}{dR} \log{p}(y^n; \mu, R)|_{R = \hat{R}^i} \bigg) \hat{R}^{i}$ <br />
where $\frac{d}{dR} \log{p}(y^n; \mu, R)|_{R = \hat{R}^i} = -\frac{1}{2} \sum_{t=1}^{n} H_{y_t}' \Big(\big(R_{y_t}^{i}\big)^{-1} - \big(R_{y_t}^{i}\big)^{-1} (y_t - \mu_{y_t}) (y_t - \mu_{y_t})' \big(R_{y_t}^{i}\big)^{-1}\Big) H_{y_t}$

2. $\hat{\mu} = \big(\sum_{t=1}^{n}H_{y_t}'R_{y_t}^{-1}H_{y_t}\big)^{-1} \big(\sum_{t=1}^{n}H_{y_t}'R_{y_t}^{-1}y_t\big)$

In [22]:
@tf.function(experimental_relax_shapes=True)
def run_graph_mcmichael(mu, R, y_t, H_yt, H_yt_trans, k_t):
    # for R estimation
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_det = tf.linalg.det(R_yt)
    R_yt_inv = tf.linalg.inv(R_yt)
    mu_yt = tf.matmul(H_yt, mu)
    log_p_gradient_part = H_yt_trans @ (R_yt_inv - R_yt_inv @ (y_t - mu_yt) @ tf.transpose(y_t - mu_yt) @ R_yt_inv) @ H_yt

    # for mu estimation
    Hyt_trans_Ryt_inv_Hyt_sum_part = H_yt_trans @ R_yt_inv @ H_yt
    Hyt_trans_Ryt_inv_yt_sum_part = H_yt_trans @ R_yt_inv @ y_t
    
    # for log likelihood calculation
    log_p_hat_part = -1/2*(tf.math.log(R_yt_det) + tf.transpose(y_t - mu_yt) @ R_yt_inv @ (y_t - mu_yt) + k_t*LOG_2PI)
    
    return log_p_gradient_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part

In [23]:
def mcmichael(mu, R):
    gamma = 0.00001
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    log_p_gradient = 0
    log_p_hat = 0

    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, y_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
        log_p_gradient_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part = run_graph_mcmichael(mu, R, y_t, H_yt, H_yt_trans, k_t)
        
        log_p_gradient += log_p_gradient_part
        Hyt_trans_Ryt_inv_Hyt_sum += Hyt_trans_Ryt_inv_Hyt_sum_part
        Hyt_trans_Ryt_inv_yt_sum += Hyt_trans_Ryt_inv_yt_sum_part
        log_p_hat += log_p_hat_part
        
    R_hat = R + gamma*(R @ (-1/2*log_p_gradient) @ R)
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)
    return mu_hat, R_hat, log_p_hat

In [24]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = tf.constant(-np.inf, dtype=tf.float32)

for i in range(40):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = mcmichael(mu, R)
    convergence_criterion = log_p_hat/n - log_p/n < delta
    
    print(f'normalized log_p_hat: {(log_p_hat/n).numpy().flatten()[0]:.5}')
    print(f'normalized log_p:     {(log_p/n).numpy().flatten()[0]:.5}')
    print(f'convergence gap:      {(log_p_hat/n - log_p/n).numpy().flatten()[0]:.5}')
    
    if convergence_criterion:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 1/137328 [00:00<4:37:27,  8.25it/s]

iteration: 0


  0%|          | 149/137328 [00:00<01:32, 1488.12it/s]

normalized log_p_hat: -32.233
normalized log_p:     -inf
convergence gap:      inf


  0%|          | 166/137328 [00:00<01:23, 1649.90it/s]

...

normalized log_p_hat: -31.416
normalized log_p:     -31.416
convergence gap:      0.00065422


  0%|          | 310/137328 [00:00<01:25, 1603.43it/s]

normalized log_p_hat: -31.415
normalized log_p:     -31.416
convergence gap:      0.00051689


100%|██████████| 137328/137328 [01:14<00:00, 1839.26it/s]

normalized log_p_hat: -31.415
normalized log_p:     -31.415
convergence gap:      0.0004673


In [25]:
# 35 iterations, ~38 min
np.save('results/mcmichael_mu.npy', mu_hat)
np.save('results/mcmichael_R.npy', R_hat)
np.save('results/mcmichael_log_p.npy', log_p_hat)

## Evaluation

$\epsilon^2 = \frac{\sum_{t=1}^{n}\big(x_t - \hat{X_t}\big)'\big(x_t - \hat{X_t}\big)}{\sum_{t=1}^{n} l_t}$

In [26]:
@tf.function(experimental_relax_shapes=True)
def run_graph_square_error(mu, R, movie_ids_t_P_data, ratings_t_P_data, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans):
    # calculate X_t_hat
    R_xt = H_xt @ R @ H_xt_trans
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_inv = tf.linalg.inv(R_yt)
    R_xtyt = H_xt @ R @ H_yt_trans

    mu_yt = tf.matmul(H_yt, mu)
    mu_xt = tf.matmul(H_xt, mu)

    X_t_hat = R_xtyt @ R_yt_inv @ (y_t - mu_yt) + mu_xt
    
    # clip ratings
    predictions_t = tf.gather(tf.matmul(H_xt_trans, X_t_hat), indices=movie_ids_t_P_data-1)
    predictions_t = tf.clip_by_value(predictions_t, 1, 5)
    
    return tf.matmul(tf.transpose(ratings_t_P_data - predictions_t), ratings_t_P_data - predictions_t)

In [27]:
def evaluate(mu, R):
    square_error = 0
    l = 0
    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, y_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
        square_error += run_graph_square_error(mu, R, movie_ids_t_P_data, ratings_t_P_data, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans)
        l += len(ratings_t_P_data)
    return np.sqrt(square_error/l)

In [28]:
em_mu = np.load('results/em_mu.npy')
em_R = np.load('results/em_R.npy')
rmse = evaluate(em_mu, em_R)
rmse

100%|██████████| 137328/137328 [01:21<00:00, 1683.28it/s]


array([[0.91699296]], dtype=float32)

In [29]:
mcmichael_mu = np.load('results/mcmichael_mu.npy')
mcmichael_R = np.load('results/mcmichael_R.npy')
rmse = evaluate(mcmichael_mu, mcmichael_R)
rmse

100%|██████████| 137328/137328 [01:21<00:00, 1692.77it/s]


array([[0.917]], dtype=float32)